# 运行环境
`diffusers == 0.121`

# Encoder 的作用
- 特征提取：
  - Encoder 负责从原始输入数据中提取高层次的特征。对于图像来说，Encoder 可能会提取出边缘、纹理等特征；对于文本来说，它可能会提取出句法结构和语义信息。这种特征提取通常通过多个卷积层（对于图像）或多个 LSTM/Transformer 层（对于文本）来实现。

- 降维和压缩：
  - Encoder 通常将高维的输入数据转换为低维的潜在表示（latent representation）。这种潜在表示保留了输入数据的最重要信息，但维度较低，有助于减少计算复杂性和存储需求。在自编码器中，编码器的输出通常是潜在空间（latent space）中的一个向量，这个向量是数据在较低维度空间中的压缩表示。

In [7]:
import torch

# 嵌入层
class Embed(torch.nn.Module):
    def __init__(self):
        super().__init__() # 因为继承了父类, 所以这里是做个初始化(🔥 能够把父类的属性继承过来)
        self.embed = torch.nn.Embedding(49408, 768)  # 创建词嵌入层，词汇表大小为 49408，嵌入维度为 768
        self.pos_embed = torch.nn.Embedding(77, 768)  # 创建位置嵌入层，位置数量为 77，嵌入维度为 768
        self.register_buffer('pos_ids', torch.arange(77).unsqueeze(dim=0))  # 注册位置索引缓冲区，范围为 [0, 76]，形状为 [1, 77]
        
    def forward(self, input_ids): # 定义前向传播方法，input_ids 是输入的词索引
        
        # 形状从 [batch_size, 77] -> [batch_size, 77, 768]
        embed = self.embed(input_ids) # ✏️ 词编码（提示词）
        
        # 对位置索引进行位置编码
        # 形状从 [1, 77] -> [batch_size, 77, 768]
        pos_embed = self.pos_embed(self.pos_ids) # 🚗 位置编码（图片）
        
        # [batch_size, 77, 768]
        # 将词嵌入和位置嵌入相加，得到最终的嵌入表示
        
        # 输出形状为 [batch_size, 77, 768]
        return embed + pos_embed
    
# 实例化 Embed 类，并对输入为 [2, 77] 的张量进行前向传播，输出张量的形状
Embed()(torch.ones(2, 77).long()).shape

torch.Size([2, 77, 768])

In [2]:

# 注意力层
class Atten(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # 多头线性
        self.q = torch.nn.Linear(768, 768)
        self.k = torch.nn.Linear(768, 768)
        self.v = torch.nn.Linear(768, 768)
        self.out = torch.nn.Linear(768, 768)

        
     # 定义向前传播的逻辑
    def forward(self, x):
        # x -> [batch_size, 77, 768]
        batch_size = x.shape[0]
        
        # 维度不变
        # [batch_size, 77, 768]
        q = self.q(x) * 0.125  # 通过查询线性层计算查询向量，并缩放，形状为 [batch_size, 77, 768]
        k = self.k(x)  # 通过键线性层计算键向量，形状为 [batch_size, 77, 768]
        v = self.v(x)  # 通过值线性层计算值向量，形状为 [batch_size, 77, 768]
        
        # 拆分注意力头 (一分为三, 抽取每个特征的片段)
        # 经典的 Transformer 模型（如 BERT、GPT）通常使用 12 个注意力头, 每个头的维度为 64
        # [batch_size, 77, 768] -> [batch_size, 77, 12, 64] -> [batch_size, 12, 77, 64] -> [batch_size * 12, 77, 64]
        q = q.reshape(batch_size, 77, 12, 64).transpose(1, 2).reshape(batch_size * 12, 77, 64)
        k = k.reshape(batch_size, 77, 12, 64).transpose(1, 2).reshape(batch_size * 12, 77, 64)
        v = v.reshape(batch_size, 77, 12, 64).transpose(1, 2).reshape(batch_size * 12, 77, 64)
        
        # 注意力计算通过矩阵乘法实现 => 计算查询与键的相似性
        # [batch_size * 12, 77, 64] @ [batch_size * 12, 64, 77] -> [batch_size * 12, 77, 77]
        attention = torch.bmm(q, k.transpose(1, 2))
        
        # 重新调整形状，返回形状为 [batch_size, 12, 77, 77] 的注意力矩阵
        attention = attention.reshape(batch_size, 12, 77, 77)
        
        
        
        # Mask (选择性地允许或阻止注意力, 向后注意力 mask, 后面只能注意到后一个) => 生成一个用于掩码（Mask）的张量，适用于自注意力机制中的序列处理        
        def get_mask(batch_size):
            mask = torch.empty(batch_size, 77, 77)

            # 上三角对角线以上的部分置为负无穷
            mask.fill_(-float('inf'))

            # 对角线和以下的位置为0
            mask.triu_(1)

            return mask.unsqueeze(1)
        
        
        # 👇 使用 mask 来遮盖注意力
        # [batch_size, 12, 77, 77] + [batch_size, 1, 77, 77] -> [batch_size, 12, 77, 77]
        attention = attention + get_mask(attention.shape[0]).to(attention.device)

        # [batch_size, 12, 77, 77] -> [batch_size * 12, 77, 77]
        attention = attention.reshape(batch_size * 12, 77, 77)

        # 计算 softmax, 被 mask 的部分值压缩为 0, 因为 负无穷, 所以每个词只能注意到前一个词
        attention = attention.softmax(dim=-1)

        # 计算和 v 的乘积
        attention = torch.bmm(attention, v)

        # [batch_size * 12, 77, 64] -> [batch_size, 12, 77, 64] -> [batch_size, 77, 768]
        attention = attention.reshape(batch_size, 12, 77, 64).transpose(1, 2).reshape(batch_size, 77, 768)

        # 线性输出, 维度不变
        # [batch_size, 77, 768]
        return self.out(attention)

Atten()(torch.randn(2, 77, 768)).shape

torch.Size([2, 77, 768])

In [3]:

# 编码器层
class ClipEncoder(torch.nn.Module):
    def __init__(self):
        super().__init__()  # 调用父类的初始化方法，继承父类的属性和方法
        
        # 👇 使用 Sequential 来  按顺序  组合多个神经网络层
        # 第一层序列化网络：包括层归一化和自定义注意力机制
        self.s1 = torch.nn.Sequential(
            torch.nn.LayerNorm(768),  # 归一化输入，保持稳定训练
            Atten(),  # 自定义注意力层，处理输入数据
        )
        
        # 第二层序列化网络：包括层归一化和全连接层
        self.s2 = torch.nn.Sequential(
            torch.nn.LayerNorm(768),  # 再次进行归一化
            torch.nn.Linear(768, 3072)  # 将输入从 768 维映射到 3072 维
        )
        
        # 第三层：线性变换，将维度从 3072 再次缩回到 768
        self.s3 = torch.nn.Linear(3072, 768)
        
        
    def forward(self, x):
        # 输入 x 的形状为 [2, 77, 768]
        # 通过第一层处理，维度保持不变 -> [2, 77, 768]
        x = x + self.s1(x)  # 残差连接（ResNet 风格），将原始输入与注意力层输出相加
        
        # 保存当前的 x 值，用于后面的残差连接
        res = x  # 保存第一层的输出，用于残差连接
        
        # 通过第二层处理，维度从 [2, 77, 768] -> [2, 77, 3072]
        x = self.s2(x)
        
        # 使用激活函数处理 x，维度不变
        x = x * (x * 1.702).sigmoid() 
        
        # 通过第三层线性变换，将维度从 [2, 77, 3072] 缩回到 [2, 77, 768]
        # 将结果与之前保存的 res 相加，形成最终输出
        return res + self.s3(x)  # 残差连接，将原始输入与处理后的输出相加
    
# 测试 ClipEncoder 类的功能
# print(ClipEncoder()(torch.randn(2, 77, 768)).shape)  # 输出张量的形状应该为 [2, 77, 768]
ClipEncoder()(torch.randn(2, 77, 768)).shape  # 输出张量的形状应该为 [2, 77, 768]

torch.Size([2, 77, 768])

In [4]:
# 主模型
encoder = torch.nn.Sequential( # 👈 使用 Sequential 来  按顺序  组合多个神经网络层
    Embed(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    torch.nn.LayerNorm(768)
)

# 👇 对模型进行试算, 预期输出 torch.Size([2, 77, 768])
encoder(torch.ones(2, 77).long()).shape

torch.Size([2, 77, 768])

In [5]:

# 👋 加载预训练的模型 ————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
from transformers import CLIPTextModel
import os


print(os.listdir('model/'))

#加载预训练模型的参数
try:
    params = CLIPTextModel.from_pretrained(
        'model/diffsion_from_scratch.params', subfolder='text_encoder')
    print("✅ Model loaded successfully!")
except Exception as e:
    print("❌ Error loading model:", e)
    
#词编码
encoder[0].embed.load_state_dict(
    params.text_model.embeddings.token_embedding.state_dict())

#位置编码
encoder[0].pos_embed.load_state_dict(
    params.text_model.embeddings.position_embedding.state_dict())

#12层编码层
for i in range(12):

    #第一层norm
    encoder[i + 1].s1[0].load_state_dict(
        params.text_model.encoder.layers[i].layer_norm1.state_dict())

    #注意力q矩阵
    encoder[i + 1].s1[1].q.load_state_dict(
        params.text_model.encoder.layers[i].self_attn.q_proj.state_dict())

    #注意力k矩阵
    encoder[i + 1].s1[1].k.load_state_dict(
        params.text_model.encoder.layers[i].self_attn.k_proj.state_dict())

    #注意力v矩阵
    encoder[i + 1].s1[1].v.load_state_dict(
        params.text_model.encoder.layers[i].self_attn.v_proj.state_dict())

    #注意力out
    encoder[i + 1].s1[1].out.load_state_dict(
        params.text_model.encoder.layers[i].self_attn.out_proj.state_dict())

    #第二层norm
    encoder[i + 1].s2[0].load_state_dict(
        params.text_model.encoder.layers[i].layer_norm2.state_dict())

    #mlp第一层fc
    encoder[i + 1].s2[1].load_state_dict(
        params.text_model.encoder.layers[i].mlp.fc1.state_dict())

    #mlp第二层fc
    encoder[i + 1].s3.load_state_dict(
        params.text_model.encoder.layers[i].mlp.fc2.state_dict())

#输出norm
encoder[13].load_state_dict(params.text_model.final_layer_norm.state_dict())


/Users/zeno/mambaforge/envs/torch_gpu_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['.DS_Store', 'diffsion_from_scratch.params', 'diffsion_from_scratch.unet']
✅ Model loaded successfully!


<All keys matched successfully>

In [6]:
# 计算模型的输出结果(我们自己的模型跟预训练的模型, 理论上输出应该是一样的)
a = encoder(torch.arange(77).unsqueeze(dim=0))
b = params(torch.arange(77).unsqueeze(dim=0)).last_hidden_state


print(a)
print(b)
# (a==b).all() # 打印应该是: True
# 比较它们是否在可接受的误差范围内相等
are_close = torch.allclose(a, b, atol=1e-4)

print("👀 两个模型的张量是否相近", are_close)  # 这应该会输出 True


# 打印差异最大值
print((a - b).abs().max())

tensor([[[-0.3488,  0.0139, -0.0409,  ..., -0.4707, -0.2910,  0.0627],
         [ 0.6009, -0.4915,  1.0705,  ...,  0.0032,  0.5970, -0.4605],
         [ 0.5848, -1.8402,  0.6390,  ...,  0.3736,  0.1611,  1.0529],
         ...,
         [ 0.7383, -0.1099,  1.2613,  ...,  0.2626, -0.2641,  0.3401],
         [ 1.1845, -0.1865,  1.5217,  ...,  0.2758,  0.1133,  0.1809],
         [ 0.9668, -0.5271,  1.4090,  ..., -0.0710,  0.1474, -0.2603]]],
       grad_fn=<NativeLayerNormBackward0>)
tensor([[[-0.3488,  0.0139, -0.0409,  ..., -0.4707, -0.2910,  0.0627],
         [ 0.6009, -0.4915,  1.0705,  ...,  0.0032,  0.5970, -0.4605],
         [ 0.5848, -1.8402,  0.6390,  ...,  0.3736,  0.1611,  1.0529],
         ...,
         [ 0.7383, -0.1099,  1.2613,  ...,  0.2626, -0.2641,  0.3401],
         [ 1.1845, -0.1865,  1.5217,  ...,  0.2758,  0.1133,  0.1809],
         [ 0.9668, -0.5271,  1.4090,  ..., -0.0710,  0.1474, -0.2603]]],
       grad_fn=<NativeLayerNormBackward0>)
👀 两个模型的张量是否相近 True
tensor(1.88